# 2장 시계열 데이터의 발견과 정리 - Python
## 1. 시계열 데이터는 어디서 찾는가

### (1) 미리 준비된 데이터셋

- UCI 머신러닝 저장소
    - 80개의 시계열 데이터셋 보유
- UEA 및 UCR 시계열 분류 저장소
    - 요가 동작 부류 작업
    - 와인 데이터셋
        - x 축의 순서로 얻는 추가정보는 시간적인 요소가 없더라도 시계열 아이디어 적용 O
- 정부 시계열 데이터셋
    - NOAA 국립환경정보센터
    - 미국 노동 통계국
    - 미국 질병 통제예방센터
    - 세인트루이스 연방준비은행
- 그 외
    - CompEngine
    - R 패키디 : Mcomp, Mc4comp2018

### (2) 발견된 시계열

: 직접 수집한 시계열 데이터

- 타임스탬프가 찍힌 이벤트 기록
- 시간을 대체하는 ‘시간이 없는’ 측정
- 물리적 흔적

In [1]:
import pandas as pd
import io
import requests

In [2]:
YearJoined = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/year_joined.csv")

In [3]:
emails = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/emails.csv")

In [4]:
donations = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/donations.csv")

In [10]:
YearJoined

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018
...,...,...,...
995,995,bronze,2016
996,996,bronze,2018
997,997,bronze,2018
998,998,bronze,2017


In [11]:
emails

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00
...,...,...,...
25483,3.0,998.0,2018-04-30 00:00:00
25484,3.0,998.0,2018-05-07 00:00:00
25485,3.0,998.0,2018-05-14 00:00:00
25486,3.0,998.0,2018-05-21 00:00:00


### (1) 작업의 예: 시계열 데이터 집합 조립하기
- 특정 회원에 대한 기록이 하나 이상인지 확인

→ 1000명의 모든 회원이 단 하나의 상태만 가진다

⇒ 값이 언제 할당되었는지 알 수 없는 변수의 사용은 지양

In [12]:
YearJoined.groupby('user').count().groupby('userStats').count()


,yearJoined
userStats,
1,1000


---
주를 나눌 때는 일-토 or 월-일로 잡기

- Null 여부도 확인하기 (특정 회원이 이메일을 오픈하지 않은 경우)

-> Null이 없는 상황

In [6]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


---
- 특정 한 회원에 대해 확인작업

-> 일부 주가 누락됨 (2017.12.18 이후부터 열람 기록X)

In [15]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04 00:00:00
25465,3.0,998.0,2017-12-11 00:00:00
25466,3.0,998.0,2017-12-18 00:00:00
25467,3.0,998.0,2018-01-01 00:00:00
25468,3.0,998.0,2018-01-08 00:00:00
25469,2.0,998.0,2018-01-15 00:00:00
25470,3.0,998.0,2018-01-22 00:00:00
25471,2.0,998.0,2018-01-29 00:00:00
25472,3.0,998.0,2018-02-05 00:00:00
25473,3.0,998.0,2018-02-12 00:00:00


---
- 특정 회원에 대한 사건이 처음 발생한 시점과 마지막에 발생한 시점 사이에 주가 몇개 기록되었는지 확인

-> 실제로는 26개의 주가 있는데 데이터 셋에는 24로 확인이 됨 

=> 누락된 주가 있다는 사실을 알 수 있음

In [13]:
import datetime
date_time_str_max = max(emails[emails.user == 998].week)
date_time_str_min = min(emails[emails.user == 998].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

25.0

In [16]:
emails[emails.user == 998].shape

(24, 3)

---
- 처음과 마지막 시기의 사잇갑승로 누락된 부분 채워넣기

팬던스의 색인 기능 활용 (MultiIndex) : 곱집합

-> 998 회원이 초반에 0이 많은데 그 이유는 가입 전이라서

In [18]:
#set을 이용하여 각 열의 유니크한 값 목록 만들기
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [19]:
#재색인 메서드: reindex
#색인 재설정 메서드 : reset_index
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()

#재설정된 색인에 의해 생성된 열의 이름을 붙여주기
all_email.columns = ['week', 'user', 'emailsOpened']

In [20]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
35034,2015-02-09 00:00:00,998.0,0.0
21020,2015-02-16 00:00:00,998.0,0.0
71686,2015-02-23 00:00:00,998.0,0.0
34495,2015-03-02 00:00:00,998.0,0.0
87317,2015-03-09 00:00:00,998.0,0.0
...,...,...,...
66296,2018-04-30 00:00:00,998.0,3.0
55516,2018-05-07 00:00:00,998.0,3.0
63601,2018-05-14 00:00:00,998.0,3.0
88934,2018-05-21 00:00:00,998.0,3.0


In [21]:
#회원이 이메일을 수신해온 주의 시작과 끝 결과 얻기
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()

In [22]:
# 0이었던 열처럼 기여가 별로 없는 행은 삭제 = 가입 하기 전의 주들은 삭제
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

### 2. 발견된 시계열을 구성하기
- 기부 데이터를 주간 시계열로 변경 -> 이메일 데이터와 비교하기 위해

In [53]:
donations

,amount,user
timestamp,,
2017-11-12 11:13:44,25.0,0.0
2015-08-25 19:01:45,50.0,0.0
2015-03-26 12:03:47,25.0,0.0
2016-07-06 12:24:55,50.0,0.0
2016-05-11 18:13:04,50.0,1.0
...,...,...
2016-09-02 11:20:00,25.0,992.0
2017-11-02 12:17:06,50.0,993.0
2016-09-13 21:09:47,1000.0,995.0


In [23]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())

In [67]:
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64

---
- 기부와 이메일 정보를 하나로 합치기

In [79]:

merged_df = pd.DataFrame()

for user, user_email in all_email.groupby('user'):
    user_donations = agg_don[agg_don.index.get_level_values('user') == user]
  
    user_donations = user_donations.droplevel(0)
  # user_donations.set_index('timestamp', inplace = True)  
  
    user_email = all_email[all_email.user == user]
    user_email.sort_values('week', inplace=True)
    user_email.set_index('week', inplace=True)

    df = pd.merge(user_email, user_donations, how='left', left_index=True, right_index=True)
    df.fillna(0)

    merged_df = merged_df.append(df.reset_index()[['user', 'week', 'emailsOpened', 'amount']])

In [80]:
merged_df.head()

,user,week,emailsOpened,amount
0,1.0,2015-06-29 00:00:00,3.0,NaN
1,1.0,2015-07-06 00:00:00,0.0,NaN
2,1.0,2015-07-13 00:00:00,2.0,NaN
3,1.0,2015-07-20 00:00:00,2.0,NaN
4,1.0,2015-07-27 00:00:00,3.0,NaN


In [92]:
merged_df[(merged_df.user == 998)&(merged_df.amount.notna())]

,user,week,emailsOpened,amount


---
- 다음 기부 예측 모델 만든다고 가정 -> 한 주의 기부금을 기부가 발생하기 전 주의 이메일 응답 방식에 따라서 정렬

=> 한주 뒤로 옮기고 싶다면 shifr 연산자 사용

In [81]:
df = merged_df[merged_df.user == 998]

#target 변수에 amount 열의 기록을 1만큼 뒤로 미룬 값을 넣기
df['target'] = df.amount.shift(1)
df = df.fillna(0)
df

,user,week,emailsOpened,amount,target
0,998.0,2017-12-04 00:00:00,1.0,0.0,0.0
1,998.0,2017-12-11 00:00:00,3.0,0.0,0.0
2,998.0,2017-12-18 00:00:00,3.0,0.0,0.0
3,998.0,2017-12-25 00:00:00,0.0,0.0,0.0
4,998.0,2018-01-01 00:00:00,3.0,0.0,0.0
5,998.0,2018-01-08 00:00:00,3.0,0.0,0.0
6,998.0,2018-01-15 00:00:00,2.0,0.0,0.0
7,998.0,2018-01-22 00:00:00,3.0,0.0,0.0
8,998.0,2018-01-29 00:00:00,2.0,0.0,0.0
9,998.0,2018-02-05 00:00:00,3.0,0.0,0.0


## 3. 타임스탬프의 문제점

## 4. 데이터 정리

### (1) 누락된 데이터 다루기
- 대치법 : 전체의 관측에 기반하여 데이터 채우기
- 보간법 : 인접한 데이터 사용
- 영향받은 기간 삭제 : 해당 기간은 완전히 사용X

#### 포워드 필 
누락된 값이 나타나기 직전의 값으로 누락된 값을 채우기
#### 이동평균 
과거의 값으로 미래의 값을 예측 (포워드 필)
#### 보간법 
전체 데이터를 기하학적인 행동에 제한하여 누락된 데이터값을 결정
- 선형 보간법 : 누락된 데이터가 주변 데이터에 선형적이 일관성을 갖도록 제한
    - 해당 추세를 작 적용할 수 잇음
    - 단, 강수량 같은 경우는 적절X
    
### (2) 업샘플링과 다운샘플링
: 타임스탬프의 빈도를 늘이거나 줄이는 방법

#### 다운샘플링
데이터의 빈도를 줄이기 (일->월)
- 원본 데이터의 시간 단위가 실용적이지 않은 경우 (너무 자주 측정해서)
- 계정 주기의 특정 부분에 집중하는 경우
- 더 낮은 빈도의 데이터에 맞추는 경우

#### 업샘플링
데이터의 빈도 높이기 (일->분)
- 시계열이 불규칙적인 상황
- 입력이 서로 다른 빈도로 샘플링된 상황

### (3) 데이터 평활
- 평활의 목적 : 측정의 오류, 높게 튀는 측정치를 제거하기 위해 이동평균(누락된 데이터를 주변값의 평균으로 대치) 사용
    - 데이터 준비
    - 특징 생성
    - 예측
    - 시각화
    
- 지수평활 : 최근 데이터일수록 더 많은 가중치를 줘서 시간의 특성을 더 잘 인식


In [28]:

air = pd.read_csv("https://raw.githubusercontent.com/PracticalTimeSeriesAnalysis/BookRepo/master/Ch02/data/AirPassengers.csv", names=['Date', 'Passengers'])

In [29]:
air

,Date,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [93]:
#함수에 감쇠요인을 적용하여 탑승객으 수의 값을 손쉽게 평활 
air['Smooth.5'] = air.ewm(alpha = .5).Passengers.mean()
air['Smooth.9'] = air.ewm(alpha = .9).Passengers.mean()

#alpha값이 클수록 현재 값에 가깝도록 더 빨리 갱신

In [31]:
air

,Date,Passengers,Smooth.5,Smooth.9
0,1949-01,112,112.000000,112.000000
1,1949-02,118,116.000000,117.454545
2,1949-03,132,125.142857,130.558559
3,1949-04,129,127.200000,129.155716
4,1949-05,121,124.000000,121.815498
...,...,...,...,...
139,1960-08,606,582.096411,606.665454
140,1960-09,508,545.048205,517.866545
141,1960-10,461,503.024103,466.686655
142,1960-11,390,446.512051,397.668665


## 5. 계절성 데이터
- 계절성 : 특정 행동의 빈도가 안정적으로 반복해서 나타남
    - 선으로 그래프 그림

## 6. 시간대

In [94]:

import datetime
datetime.datetime.utcnow()

datetime.datetime(2024, 7, 22, 10, 58, 31, 358245)

In [95]:
datetime.datetime.now()

# => 시간대를 부여하지 않으면 UTC를 반환하지 않음

datetime.datetime(2024, 7, 22, 19, 59, 4, 926789)

In [34]:
datetime.datetime.now(datetime.timezone.utc)

datetime.datetime(2024, 7, 22, 10, 10, 24, 731847, tzinfo=datetime.timezone.utc)

In [35]:
#미서부는 태평양 표준시를 지정할 수 있음

import pytz

western = pytz.timezone('US/Pacific')
western.zone

'US/Pacific'

In [36]:
#시간대를 현지화하기
loc_dt = western.localize(datetime.datetime(2018, 5, 15, 12, 34, 0))
loc_dt

datetime.datetime(2018, 5, 15, 12, 34, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [37]:
#tzinfo를 사용하면 항상 원하는 결과를 얻게되는 것은 아님 
london_tz = pytz.timezone('Europe/London')
london_dt = loc_dt.astimezone(london_tz)
london_dt

datetime.datetime(2018, 5, 15, 20, 34, tzinfo=<DstTzInfo 'Europe/London' BST+1:00:00 DST>)

In [38]:
f = '%Y-%m-%d %H:%M:%S %Z%z'
datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz).strftime(f)

'2018-05-12 12:15:00 LMT-0001'

In [39]:
event1 = datetime.datetime(2018, 5, 12, 12, 15, 0, tzinfo = london_tz)
event2 = datetime.datetime(2018, 5, 13, 9, 15, 0, tzinfo = western)
event2 - event1

datetime.timedelta(days=1, seconds=17520)

In [40]:
event1 = london_tz.localize(datetime.datetime(2018, 5, 12, 12, 15, 0))
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0))
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [41]:
event1 = london_tz.localize((datetime.datetime(2018, 5, 12, 12, 15, 0))).astimezone(datetime.timezone.utc)
event2 = western.localize(datetime.datetime(2018, 5, 13, 9, 15, 0)).astimezone(datetime.timezone.utc)
event2 - event1

datetime.timedelta(days=1, seconds=18000)

In [42]:
pytz.common_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

In [43]:
pytz.country_timezones('RU')

['Europe/Kaliningrad',
 'Europe/Moscow',
 'Europe/Kirov',
 'Europe/Volgograd',
 'Europe/Astrakhan',
 'Europe/Saratov',
 'Europe/Ulyanovsk',
 'Europe/Samara',
 'Asia/Yekaterinburg',
 'Asia/Omsk',
 'Asia/Novosibirsk',
 'Asia/Barnaul',
 'Asia/Tomsk',
 'Asia/Novokuznetsk',
 'Asia/Krasnoyarsk',
 'Asia/Irkutsk',
 'Asia/Chita',
 'Asia/Yakutsk',
 'Asia/Khandyga',
 'Asia/Vladivostok',
 'Asia/Ust-Nera',
 'Asia/Magadan',
 'Asia/Sakhalin',
 'Asia/Srednekolymsk',
 'Asia/Kamchatka',
 'Asia/Anadyr']

In [44]:
pytz.country_timezones('fr')

['Europe/Paris']

In [45]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00)).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)

In [46]:
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [47]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [48]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 2, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

In [49]:
ambig_time = western.localize(datetime.datetime(2002, 10, 27, 1, 30, 00), is_dst = True).astimezone(datetime.timezone.utc)
ambig_time_earlier = ambig_time - datetime.timedelta(hours=1)
ambig_time_later = ambig_time + datetime.timedelta(hours=1)

In [50]:
ambig_time_earlier.astimezone(western)

datetime.datetime(2002, 10, 27, 0, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [51]:
ambig_time.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [52]:
ambig_time_later.astimezone(western)

datetime.datetime(2002, 10, 27, 1, 30, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>)

## 7. 사전관찰의 방지


1. 누락된 데이터를 대치하거나 평활시 : 사전관찰의 도입이 결과에 어떤 영향을 주는지 확인
2. 매우 적은 데이터셋으로 처리의 전체 공정 구축
3. 타임스탬프와 관련된 지연이 무엇인지 확인
4. 시간을 인식할 수 있는 에러 검사 또는 교차검증
5. 의도적으로 사전관찰 도입하여 모델으 동작 확인
6. 특징을 천천히 추가하여 성능의 큰 향상이 있는지 찾아보기